In [39]:
import numpy as np 
import requests
import sqlite3
import xmltodict 

In [40]:
response = requests.get('http://knesset.gov.il/Odata/ParliamentInfo.svc/KNS_BillInitiator')
dict_data = xmltodict.parse(response.content)
dict_data['feed']['entry']

[{'id': 'http://knesset.gov.il/Odata/ParliamentInfo.svc/KNS_BillInitiator(1)',
  'category': {'@term': 'KnessetDataModel.KNS_BillInitiator',
   '@scheme': 'http://schemas.microsoft.com/ado/2007/08/dataservices/scheme'},
  'link': [{'@rel': 'edit',
    '@title': 'KNS_BillInitiator',
    '@href': 'KNS_BillInitiator(1)'},
   {'@rel': 'http://schemas.microsoft.com/ado/2007/08/dataservices/related/KNS_Bill',
    '@type': 'application/atom+xml;type=entry',
    '@title': 'KNS_Bill',
    '@href': 'KNS_BillInitiator(1)/KNS_Bill'},
   {'@rel': 'http://schemas.microsoft.com/ado/2007/08/dataservices/related/KNS_Person',
    '@type': 'application/atom+xml;type=entry',
    '@title': 'KNS_Person',
    '@href': 'KNS_BillInitiator(1)/KNS_Person'}],
  'title': None,
  'updated': '2022-05-23T14:26:43Z',
  'author': {'name': None},
  'content': {'@type': 'application/xml',
   'm:properties': {'d:BillInitiatorID': {'@m:type': 'Edm.Int32',
     '#text': '1'},
    'd:BillID': {'@m:type': 'Edm.Int32', '#text'

In [41]:
db = sqlite3.connect('KNESSET.db')

In [42]:
cursor = db.cursor()
# cursor.execute("DROP TABLE KNS_BillInitiator")

cursor.execute('''
    CREATE TABLE KNS_BillInitiator(
        BillInitiatorID INTEGER PRIMARY KEY, 
        BillID INTEGER,
        PersonID INTEGER, 
        IsInitiator BLOB, 
        Ordinal INTEGER,
        LastUpdatedDate TEXT)
''')
db.commit()

In [ ]:
rows = []
for elem in dict_data['feed']['entry']:
    row = elem['content']['m:properties']
    BillInitiatorID = int(row['d:BillInitiatorID']['#text'])
    BillID = int(row['d:BillID']['#text'])
    PersonID = int(row['d:PersonID']['#text'])
    IsInitiator = 1 if row['d:IsInitiator']['#text'] == 'true' else 0
    Ordinal = int(row['d:Ordinal']['#text'])
    LastUpdatedDate = row['d:LastUpdatedDate']['#text']

    rows.append((BillID, PersonID, IsInitiator, Ordinal, LastUpdatedDate))


In [44]:
cursor.executemany('''
    INSERT INTO KNS_BillInitiator(BillID, PersonID, IsInitiator, Ordinal, LastUpdatedDate)
    VALUES(?,?,?,?,?)
''', rows)

In [45]:
cursor.execute('''
    SELECT * FROM KNS_BillInitiator
''')

In [46]:
for row in cursor:
    print(row)

(1, 29803, 1047, 1, 7, '2015-03-20T12:03:24')
(2, 29804, 1047, 1, 2, '2015-03-20T12:03:24')
(3, 29805, 1047, 1, 3, '2015-03-20T12:03:24')
(4, 29829, 1047, 1, 4, '2015-03-20T12:03:24')
(5, 29830, 1047, 1, 2, '2015-03-20T12:03:24')
(6, 30035, 1047, 1, 8, '2015-03-20T12:03:24')
(7, 30061, 1047, 1, 2, '2015-03-20T12:03:24')
(8, 30070, 1047, 1, 17, '2015-03-20T12:03:24')
(9, 30501, 1047, 1, 6, '2015-03-20T12:03:24')
(10, 30534, 1047, 1, 3, '2015-03-20T12:03:24')
(11, 30615, 1047, 1, 1, '2015-03-20T12:03:24')
(12, 75146, 1047, 1, 7, '2015-03-20T12:03:24')
(13, 75281, 1047, 1, 7, '2015-03-20T12:03:24')
(14, 31713, 1047, 1, 7, '2015-03-20T12:03:24')
(15, 31718, 1047, 1, 1, '2015-03-20T12:03:24')
(16, 75020, 1047, 1, 8, '2015-03-20T12:03:24')
(17, 75085, 1047, 1, 2, '2015-03-20T12:03:24')
(18, 75280, 1047, 1, 2, '2015-03-20T12:03:24')
(19, 75353, 1047, 1, 2, '2015-03-20T12:03:24')
(20, 77194, 1047, 1, 7, '2015-03-20T12:03:24')
(21, 77758, 1047, 1, 5, '2015-03-20T12:03:24')
(22, 77774, 1047, 1, 

In [47]:
db.close()